In [3]:
import numpy as np
from minitorch.torch.tensor import Tensor
from minitorch.layers.layers import Linear, Sequential, Dropout
from minitorch.activations.activations import ReLU, Softmax
from numpy.typing import NDArray

In [4]:
def create(array:NDArray[np.float32]):
    return Tensor(array)

np_array = np.array([1,2,3])
x = create(np_array)

In [5]:
_original_init = Tensor.__init__

def gradient_aware_init(self, data, requires_grad=True):
    _original_init(self, data)
    self.requires_grad = requires_grad
    self.grad = None
    
Tensor.__init__ = gradient_aware_init

In [9]:
getattr(x, 'requires_grad', False) if isinstance(x, Tensor) else False

False

In [10]:
original_add = Tensor.__add__

In [12]:
y = Tensor(np.array([2,3,5]))

In [14]:
x + y

Tensor(data=[3. 5. 8.], shape=(3,), grad_info=requires_grad=True)

In [16]:
original_add(x,y)

Tensor(data=[3. 5. 8.], shape=(3,), grad_info=requires_grad=True)

In [ ]:
def ensure_grad_attr(tensor: Tensor):
    if isinstance(tensor, Tensor):
        if not hasattr(tensor, 'requires_grad'):
            tensor.requires_grad = False
        if not hasattr(tensor, 'grad'):
            tensor.grad = None
            
def _requires_grad(tensor):
    return getattr(tensor, 'requires_grad', False) if isinstance(tensor, Tensor) else False

def tracked_add(self, other):
    #* ensure the tensor has grad attributes
    if not isinstance(self, Tensor):
        self = Tensor(self, requires_grad= True)
    
    #* convert sclaler to tensor
    if not isinstance(other, Tensor):
        other = Tensor(other, requires_grad= True)

    
    #* call original add operation
    _original_add = Tensor.__add__
    result = _original_add(self, other)
    ensure_grad_attr(result)
    
    #* track gradients if needed
    if _requires_grad(self) or _requires_grad(other):
        result.requires_grad = True
        result.grad_fn = Add
    